In [1]:
import numpy as np
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error

For Q4 we used variational quantum linear solver (VQLS). The 41 by 41 matrix and the 41 by 1 vector is too large for our home computers to handle and our cicuit size goes off the charts so we using the code we submitted for Q2 we used nx=8 instead of nx=41 to make a 8 by 8 matrix and a 8 by 1 vector. The 8 by 8 matrix is donoted as A and the vector is deonted by b. 

The A matrix can be representated as a linear combination of unitary matrices A_0, A_1, A_2.... with complex coefficients C_0, C_1, C_2.......

and the vector B is repesented as a set of gate operations on n qubits. 
 So the matrix and vector that we get from the changed code of Q2 must be decomposed into the above mentioned format to be able to be given as input into our VQLS algorithm. To run the VQLS algorithm we have used pennylane with qiskit as plugin. 
 
The decomposition code is given below:

In [2]:
import itertools
import numpy as np
np.set_printoptions(precision=3)

def PauliOperator(label):
    """Generates a specified Pauli operator
    
    Parameters:
        label (str): Pauli operator label (e.g. 'ZZZZIIIX')
        
    Returns:
        numpy matrix of corresponding Pauli operator
    """
    pauli = {
        'I': np.matrix([[1,0],[0,1]]),
        'Z': np.matrix([[1,0],[0,-1]]),
        'X': np.matrix([[0,1],[1,0]]),
        'Y': np.matrix([[0,-1j],[1j,0]])
    }
    
    operator = pauli[label[0]]
    for letter in label[1:]:
        operator = np.kron(operator, pauli[letter])

    return operator

def PauliDecompose(hmat):
    """Decompose a Hermitian matrix into a sum of Pauli matrices
    
    Parameters:
        hmat (matrix): hermitian matrix to decompose
        
    Returns:
        dictionary of {Pauli matrix(str) : coefficient (float)}
    """
    coeff = {}
    nbits = int(np.log2(hmat.shape[0]))
    labels = itertools.product('IXY', repeat=nbits)
    labels = [''.join(i) for i in labels]
    for label in labels:
        tmp = np.matmul(hmat, PauliOperator(label))
        coeff[label] = np.trace(tmp) / hmat.shape[0]
    
    return coeff

M=np.array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -2.,  3.]])

PauliDecompose(M)

{'III': 1.25,
 'IIX': -0.25,
 'IIY': 1j,
 'IXI': 0.0,
 'IXX': 0.0,
 'IXY': -0.5j,
 'IYI': 0j,
 'IYX': 0.5j,
 'IYY': 0j,
 'XII': 0.0,
 'XIX': 0.0,
 'XIY': 0j,
 'XXI': 0.0,
 'XXX': 0.0,
 'XXY': -0.25j,
 'XYI': 0j,
 'XYX': -0.25j,
 'XYY': 0j,
 'YII': 0j,
 'YIX': 0j,
 'YIY': 0j,
 'YXI': 0j,
 'YXX': 0.25j,
 'YXY': 0j,
 'YYI': 0j,
 'YYX': 0j,
 'YYY': -0.25j}

We know have our 27 complex coefficients for matrix A and also we have the required gates transformation for vector b. We will now prepare the framework for the VQLS algorithm using Pennylane with Qiskit as plugin. We will call qiskit.aer as a simulator. 

In [ ]:
# Pennylane
import pennylane as qml
from pennylane import numpy as np

# Plotting
import matplotlib.pyplot as plt

############################################################################################


n_qubits = 27  # Number of system qubits.
n_shots = 10 ** 9  # Number of quantum measurements.
tot_qubits = n_qubits + 1  # Addition of an ancillary qubit.
ancilla_idx = n_qubits  # Index of the ancillary qubit (last position).
steps = 30  # Number of optimization steps
eta = 0.8  # Learning rate
q_delta = 0.001  # Initial spread of random quantum weights
rng_seed = 0  # Seed for random number generator

############################################################################################

# Coefficients of the linear combination A = c_0 A_0 + c_1 A_1 ...
c = np.array([ 1.25, -0.25, 1j, 0.0, 0.0, -0.5j, 0j, 0.5j, 0j, 0.0, 0.0, 0j, 0.0, 0.0, -0.25j, 0j, -0.25j, 0j, 0j, 0j, 0j, 0j, 0.25j, 0j, 0j, 0j, -0.25j])

def U_b():
    """Unitary matrix rotating the ground state to the problem vector |b> = U_b |0>."""
    for idx in range(n_qubits):
        if idx == 0:
 
           None
        elif idx == 1:
            qml.PauliX(wires=[0])
        elif idx == 2:
            qml.PauliY(wires=[2])
        elif idx == 3:
            qml.PauliX(wires=[3])
        elif idx == 4:
            qml.PauliX(wires=[4])
            qml.PauliX(wires=[4])
        elif idx == 5:
            qml.PauliX(wires=[5])
            qml.PauliY(wires=[5])
        elif idx == 6:
            qml.PauliY(wires=[6])
        elif idx == 7:
            qml.PauliY(wires=[7])
            qml.PauliX(wires=[7])
        elif idx == 8:
            qml.PauliY(wires=[8])
            qml.PauliY(wires=[8])
        elif idx == 9:
            qml.PauliX(wires=[9])
        elif idx == 10:
            qml.PauliX(wires=[10])
            qml.PauliX(wires=[10])
        elif idx == 11:
            qml.PauliX(wires=[11])
            qml.PauliY(wires=[11])
        elif idx == 12:
            qml.PauliX(wires=[12])
            qml.PauliX(wires=[12])
        elif idx == 13:
            qml.PauliX(wires=[13])
            qml.PauliX(wires=[13])
            qml.PauliX(wires=[13])
        elif idx == 14:
            qml.PauliX(wires=[14])
            qml.PauliX(wires=[14])
            qml.PauliY(wires=[14])
        elif idx == 15:
            qml.PauliX(wires=[15])
            qml.PauliY(wires=[15])
        elif idx == 16:
            qml.PauliX(wires=[16])
            qml.PauliY(wires=[16])
            qml.PauliX(wires=[16])
        elif idx == 17:
            qml.PauliX(wires=[17])
            qml.PauliY(wires=[17])
            qml.PauliY(wires=[17])
        elif idx == 18:
            qml.PauliY(wires=[18])
        elif idx == 19:
            qml.PauliY(wires=[19])
            qml.PauliX(wires=[19])
        elif idx == 20:
            qml.PauliY(wires=[20])
            qml.PauliY(wires=[20])
        elif idx == 21:
            qml.PauliY(wires=[21])
            qml.PauliX(wires=[21])
        elif idx == 22:
            qml.PauliY(wires=[22])
            qml.PauliX(wires=[22])
            qml.PauliX(wires=[22])
        elif idx == 23:
            qml.PauliY(wires=[23])
            qml.PauliX(wires=[23])
            qml.PauliY(wires=[23])
        elif idx == 24:
            qml.PauliY(wires=[24])
            qml.PauliY(wires=[24])
        elif idx == 25:
            qml.PauliY(wires=[25])
            qml.PauliY(wires=[25])
            qml.PauliX(wires=[25])
        elif idx == 26:
            qml.PauliY(wires=[26])
            qml.PauliY(wires=[26])
            qml.PauliY(wires=[26])     

def CA(idx):
    """Controlled versions of the unitary components A_l of the problem matrix A."""
  
    if idx == 0:
        # Identity operation
        None
    elif idx == 1:
        qml.CX(wires=[ancilla_idx, 0])
    elif idx == 2:
        qml.CY(wires=[ancilla_idx, 0])
    elif idx == 3:
        qml.CX(wires=[ancilla_idx, 0])
    elif idx == 4:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
    elif idx == 5:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
    elif idx == 6:
        qml.CY(wires=[ancilla_idx, 0])
    elif idx == 7:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
    elif idx == 8:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
    elif idx == 9:
        qml.CX(wires=[ancilla_idx, 0])
    elif idx == 10:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
    elif idx == 11:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
    elif idx == 12:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
    elif idx == 13:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
        qml.CX(wires=[ancilla_idx, 2])
    elif idx == 14:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
        qml.CY(wires=[ancilla_idx, 2])
    elif idx == 15:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
    elif idx == 16:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
        qml.CX(wires=[ancilla_idx, 2])
    elif idx == 17:
        qml.CX(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
        qml.CY(wires=[ancilla_idx, 2])
    elif idx == 18:
        qml.CY(wires=[ancilla_idx, 0])
    elif idx == 19:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
    elif idx == 20:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
    elif idx == 21:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
    elif idx == 22:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
        qml.CX(wires=[ancilla_idx, 2])
    elif idx == 23:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CX(wires=[ancilla_idx, 1])
        qml.CY(wires=[ancilla_idx, 2])
    elif idx == 24:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
    elif idx == 25:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
        qml.CX(wires=[ancilla_idx, 2])
    elif idx == 26:
        qml.CY(wires=[ancilla_idx, 0])
        qml.CY(wires=[ancilla_idx, 1])
        qml.CY(wires=[ancilla_idx, 2])

############################################################################################
def variational_block(weights):
    """Variational circuit mapping the ground state |0> to the ansatz state |x>."""
    # We first prepare an equal superposition of all the states of the computational basis.
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)

    # A very minimal variational circuit.
    for idx, element in enumerate(weights):
        qml.RY(element, wires=idx)
        
############################################################################################
dev_mu = qml.device('qiskit.aer', wires=tot_qubits)

@qml.qnode(dev_mu)
def local_hadamard_test(weights, l=None, lp=None, j=None, part=None):

    # First Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # For estimating the imaginary part of the coefficient "mu", we must add a "-i"
    # phase gate.
    if part == "Im" or part == "im":
        qml.PhaseShift(-np.pi / 2, wires=ancilla_idx)

    # Variational circuit generating a guess for the solution vector |x>
    variational_block(weights)

    # Controlled application of the unitary component A_l of the problem matrix A.
    CA(l)

    # Adjoint of the unitary U_b associated to the problem vector |b>.
    # In this specific example Adjoint(U_b) = U_b.
    U_b()

    # Controlled Z operator at position j. If j = -1, apply the identity.
    if j != -1:
        qml.CZ(wires=[ancilla_idx, j])

    # Unitary U_b associated to the problem vector |b>.
    U_b()

    # Controlled application of Adjoint(A_lp).
    # In this specific example Adjoint(A_lp) = A_lp.
    CA(lp)

    # Second Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # Expectation value of Z for the ancillary qubit.
    return qml.expval(qml.PauliZ(wires=ancilla_idx))
        
############################################################################################

def mu(weights, l=None, lp=None, j=None):
    """Generates the coefficients to compute the "local" cost function C_L."""

    mu_real = local_hadamard_test(weights, l=l, lp=lp, j=j, part="Re")
    mu_imag = local_hadamard_test(weights, l=l, lp=lp, j=j, part="Im")

    return mu_real + 1.0j * mu_imag
############################################################################################
def psi_norm(weights):
    """Returns the normalization constant <psi|psi>, where |psi> = A |x>."""
    norm = 0.0

    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            norm = norm + c[l] * np.conj(c[lp]) * mu(weights, l, lp, -1)

    return abs(norm)
############################################################################################
def cost_loc(weights):
    """Local version of the cost function. Tends to zero when A|x> is proportional to |b>."""
    mu_sum = 0.0

    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            for j in range(0, n_qubits):
                mu_sum = mu_sum + c[l] * np.conj(c[lp]) * mu(weights, l, lp, j)

    mu_sum = abs(mu_sum)

    # Cost function C_L
    return 0.5 - 0.5 * mu_sum / (n_qubits * psi_norm(weights))
############################################################################################
np.random.seed(rng_seed)
w = q_delta * np.random.randn(n_qubits)
############################################################################################

opt = qml.GradientDescentOptimizer(eta)

############################################################################################
cost_history = []
for it in range(steps):
    w = opt.step(cost_loc, w)
    cost = cost_loc(w)
    print("Step {:3d}       Cost_L = {:9.7f}".format(it, cost))
    cost_history.append(cost)

############################################################################################

plt.style.use("seaborn")
plt.plot(cost_history, "g")
plt.ylabel("Cost function")
plt.xlabel("Optimization steps")
plt.show()
############################################################################################
dev_x = qml.device('qiskit.aer', wires=n_qubits, shots=n_shots)

@qml.qnode(dev_x)
def prepare_and_sample(weights):

    # Variational circuit generating a guess for the solution vector |x>
    variational_block(weights)

    # We assume that the system is measured in the computational basis.
    # If we label each basis state with a decimal integer j = 0, 1, ... 2 ** n_qubits - 1,
    # this is equivalent to a measurement of the following diagonal observable.
    basis_obs = qml.Hermitian(np.diag(range(2 ** n_qubits)), wires=range(n_qubits))

    return qml.sample(basis_obs)

############################################################################################
samples = prepare_and_sample(w).astype(int)
q_probs = np.bincount(samples) / n_shots



############################################################################################

print("|<x|n>|^2=\n", q_probs)



############################################################################################

fig, (ax1) = plt.subplots(1, 2, figsize=(7, 4))
ax1.bar(np.arange(0, 2 ** n_qubits), q_probs, color="green")
ax1.set_xlim(-0.5, 2 ** n_qubits - 0.5)
ax1.set_xlabel("Hilbert space basis")
ax1.set_title("Quantum probabilities")

plt.show()

############################################################################################

In [ ]:
# Ideal simulator and execution
ideal_simulator = QasmSimulator()
job = execute(qml, ideal_simulator)
result_ideal = job.result()
plot_histogram(result_ideal.get_counts(0))

In [ ]:
# Example error probabilities
p_reset = 0.03
p_meas = 0.1
p_gate1 = 0.05

# QuantumError objects
error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
error_gate2 = error_gate1.tensor(error_gate1)

# Add errors to noise model
noise_bit_flip = NoiseModel()
noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

print(noise_bit_flip)

In [ ]:
# Run the noisy simulation
noisy_simulator = QasmSimulator(noise_model=noise_bit_flip)
job = execute(qml, noisy_simulator)
result_bit_flip = job.result()
counts_bit_flip = result_bit_flip.get_counts(0)

# Plot noisy output
plot_histogram(counts_bit_flip)

In [ ]:
# T1 and T2 values for qubits 0-27
T1s = np.random.normal(50e3, 10e3, 28) # Sampled from normal distribution mean 50 microsec
T2s = np.random.normal(70e3, 10e3, 28)  # Sampled from normal distribution mean 50 microsec

# Truncate random T2s <= T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(28)])

# Instruction times (in nanoseconds)
time_u1 = 0   # virtual gate
time_u2 = 50  # (single X90 pulse)
time_u3 = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond

# QuantumError objects
errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]

# Add errors to noise model
noise_thermal = NoiseModel()
for j in range(28):
    noise_thermal.add_quantum_error(errors_reset[j], "reset", [j])
    noise_thermal.add_quantum_error(errors_measure[j], "measure", [j])
    noise_thermal.add_quantum_error(errors_u1[j], "u1", [j])
    noise_thermal.add_quantum_error(errors_u2[j], "u2", [j])
    noise_thermal.add_quantum_error(errors_u3[j], "u3", [j])
    for k in range(28):
        noise_thermal.add_quantum_error(errors_cx[j][k], "cx", [j, k])

print(noise_thermal)

In [ ]:
# Run the noisy simulation
thermal_simulator = QasmSimulator(noise_model=noise_thermal)
job = execute(qml, thermal_simulator)
result_thermal = job.result()
counts_thermal = result_thermal.get_counts(0)

# Plot noisy output
plot_histogram(counts_thermal)